In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Using pre-trained word embeddings

**Author:** [fchollet](https://twitter.com/fchollet)<br>
**Date created:** 2020/05/05<br>
**Last modified:** 2020/05/05<br>
**Description:** Text classification on the Newsgroup20 dataset using pre-trained GloVe word embeddings.

## Setup

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras

## Introduction

In this example, we show how to train a text classification model that uses pre-trained
word embeddings.

We'll work with the Newsgroup20 dataset, a set of 20,000 message board messages
belonging to 20 different topic categories.

For the pre-trained word embeddings, we'll use
[GloVe embeddings](http://nlp.stanford.edu/projects/glove/).

## Download the Newsgroup20 data

In [ ]:
data_path = keras.utils.get_file(
    "news20.tar.gz",
    "http://www.cs.cmu.edu/afs/cs.cmu.edu/project/theo-20/www/data/news20.tar.gz",
    untar=True,
)

17334272/17329808 [==============================] - 9s 1us/step


In [ ]:
cd /root/.keras/datasets/20_newsgroup

/root/.keras/datasets/20_newsgroup


## Let's take a look at the data

In [ ]:
import os
import pathlib

data_dir = pathlib.Path(data_path).parent / "20_newsgroup"
dirnames = os.listdir(data_dir)
print("Number of directories:", len(dirnames))
print("Directory names:", dirnames)

fnames = os.listdir(data_dir / "comp.graphics")
print("Number of files in comp.graphics:", len(fnames))
print("Some example filenames:", fnames[:5])

Number of directories: 20
Directory names: ['talk.religion.misc', 'sci.space', 'rec.autos', 'comp.graphics', 'comp.os.ms-windows.misc', 'talk.politics.guns', 'sci.crypt', 'alt.atheism', 'talk.politics.mideast', 'comp.sys.ibm.pc.hardware', 'misc.forsale', 'rec.motorcycles', 'comp.sys.mac.hardware', 'rec.sport.baseball', 'soc.religion.christian', 'sci.electronics', 'sci.med', 'rec.sport.hockey', 'comp.windows.x', 'talk.politics.misc']
Number of files in comp.graphics: 1000
Some example filenames: ['39641', '38460', '39060', '38283', '38713']


Here's a example of what one file contains:

In [ ]:
print(open(data_dir / "comp.graphics" / "38987").read())

Newsgroups: comp.graphics
Path: cantaloupe.srv.cs.cmu.edu!das-news.harvard.edu!noc.near.net!howland.reston.ans.net!agate!dog.ee.lbl.gov!network.ucsd.edu!usc!rpi!nason110.its.rpi.edu!mabusj
From: mabusj@nason110.its.rpi.edu (Jasen M. Mabus)
Subject: Looking for Brain in CAD
Message-ID: <c285m+p@rpi.edu>
Nntp-Posting-Host: nason110.its.rpi.edu
Reply-To: mabusj@rpi.edu
Organization: Rensselaer Polytechnic Institute, Troy, NY.
Date: Thu, 29 Apr 1993 23:27:20 GMT
Lines: 7

Jasen Mabus
RPI student

	I am looking for a hman brain in any CAD (.dxf,.cad,.iges,.cgm,etc.) or picture (.gif,.jpg,.ras,etc.) format for an animation demonstration. If any has or knows of a location please reply by e-mail to mabusj@rpi.edu.

Thank you in advance,
Jasen Mabus  



Let's get rid of the headers:

In [ ]:
samples = []
labels = []
class_names = []
class_index = 0
for dirname in sorted(os.listdir(data_dir)):
    class_names.append(dirname)
    dirpath = data_dir / dirname
    fnames = os.listdir(dirpath)
    print("Processing %s, %d files found" % (dirname, len(fnames)))
    for fname in fnames:
        fpath = dirpath / fname
        f = open(fpath, encoding="latin-1")
        content = f.read()
        lines = content.split("\n")
        lines = lines[10:]
        content = "\n".join(lines)
        samples.append(content)
        labels.append(class_index)
    class_index += 1

print("Classes:", class_names)
print("Number of samples:", len(samples))

Processing alt.atheism, 1000 files found
Processing comp.graphics, 1000 files found
Processing comp.os.ms-windows.misc, 1000 files found
Processing comp.sys.ibm.pc.hardware, 1000 files found
Processing comp.sys.mac.hardware, 1000 files found
Processing comp.windows.x, 1000 files found
Processing misc.forsale, 1000 files found
Processing rec.autos, 1000 files found
Processing rec.motorcycles, 1000 files found
Processing rec.sport.baseball, 1000 files found
Processing rec.sport.hockey, 1000 files found
Processing sci.crypt, 1000 files found
Processing sci.electronics, 1000 files found
Processing sci.med, 1000 files found
Processing sci.space, 1000 files found
Processing soc.religion.christian, 997 files found
Processing talk.politics.guns, 1000 files found
Processing talk.politics.mideast, 1000 files found
Processing talk.politics.misc, 1000 files found
Processing talk.religion.misc, 1000 files found
Classes: ['alt.atheism', 'comp.graphics', 'comp.os.ms-windows.misc', 'comp.sys.ibm.pc.ha

There's actually one category that doesn't have the expected number of files, but the
difference is small enough that the problem remains a balanced classification problem.

## Shuffle and split the data into training & validation sets

In [ ]:
# Shuffle the data
seed = 1337
rng = np.random.RandomState(seed)
rng.shuffle(samples)
rng = np.random.RandomState(seed)
rng.shuffle(labels)

# Extract a training & validation split
validation_split = 0.2
num_validation_samples = int(validation_split * len(samples))
train_samples = samples[:-num_validation_samples]
val_samples = samples[-num_validation_samples:]
train_labels = labels[:-num_validation_samples]
val_labels = labels[-num_validation_samples:]

## Create a vocabulary index

Let's use the `TextVectorization` to index the vocabulary found in the dataset.
Later, we'll use the same layer instance to vectorize the samples.

Our layer will only consider the top 20,000 words, and will truncate or pad sequences to
be actually 200 tokens long.

In [ ]:
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

vectorizer = TextVectorization(max_tokens=20000, output_sequence_length=200)
text_ds = tf.data.Dataset.from_tensor_slices(train_samples).batch(128)
vectorizer.adapt(text_ds)

You can retrieve the computed vocabulary used via `vectorizer.get_vocabulary()`. Let's
print the top 5 words:

In [ ]:
vectorizer.get_vocabulary()[:5]

['', '[UNK]', 'the', 'to', 'of']

Let's vectorize a test sentence:

In [ ]:
output = vectorizer([["the cat sat on the mat"]])
output.numpy()[0, :6]

array([   2, 3546, 1816,   15,    2, 5939])

As you can see, "the" gets represented as "2". Why not 0, given that "the" was the first
word in the vocabulary? That's because index 0 is reserved for padding and index 1 is
reserved for "out of vocabulary" tokens.

Here's a dict mapping words to their indices:

In [ ]:
voc = vectorizer.get_vocabulary()
word_index = dict(zip(voc, range(len(voc))))

As you can see, we obtain the same encoding as above for our test sentence:

In [ ]:
test = ["the", "cat", "sat", "on", "the", "mat"]
[word_index[w] for w in test]

[2, 3546, 1816, 15, 2, 5939]

## Load pre-trained word embeddings

Let's download pre-trained GloVe embeddings (a 822M zip file).

You'll need to run the following commands:

```
!wget http://nlp.stanford.edu/data/glove.6B.zip
!unzip -q glove.6B.zip
```

The archive contains text-encoded vectors of various sizes: 50-dimensional,
100-dimensional, 200-dimensional, 300-dimensional. We'll use the 100D ones.

Let's make a dict mapping words (strings) to their NumPy vector representation:

In [ ]:
path_to_glove_file = "/content/drive/MyDrive/Colab Notebooks/NLPmodule/glove.6B.100d.txt"

embeddings_index = {}
with open(path_to_glove_file) as f:
    for line in f:
        word, coefs = line.split(maxsplit=1)
        coefs = np.fromstring(coefs, "f", sep=" ")
        embeddings_index[word] = coefs

print("Found %s word vectors." % len(embeddings_index))

Found 400000 word vectors.


Now, let's prepare a corresponding embedding matrix that we can use in a Keras
`Embedding` layer. It's a simple NumPy matrix where entry at index `i` is the pre-trained
vector for the word of index `i` in our `vectorizer`'s vocabulary.

In [ ]:
num_tokens = len(voc) + 2
embedding_dim = 100
hits = 0
misses = 0

# Prepare embedding matrix
embedding_matrix = np.zeros((num_tokens, embedding_dim))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # Words not found in embedding index will be all-zeros.
        # This includes the representation for "padding" and "OOV"
        embedding_matrix[i] = embedding_vector
        hits += 1
    else:
        misses += 1
print("Converted %d words (%d misses)" % (hits, misses))


Converted 17955 words (2045 misses)


Next, we load the pre-trained word embeddings matrix into an `Embedding` layer.

Note that we set `trainable=False` so as to keep the embeddings fixed (we don't want to
update them during training).

In [ ]:
from tensorflow.keras.layers import Embedding

embedding_layer = Embedding(
    num_tokens,
    embedding_dim,
    embeddings_initializer=keras.initializers.Constant(embedding_matrix),
    trainable=False,
)

## Build the model

Logistic regression

In [ ]:
from tensorflow.keras import layers

int_sequences_input = keras.Input(shape=(200,), dtype="int64")
embedded_sequences = embedding_layer(int_sequences_input)
x = layers.Flatten()(embedded_sequences)
preds = layers.Dense(len(class_names), activation="softmax", name='OutputLayer')(x)
logReg = keras.Model(int_sequences_input, preds)
logReg.summary()

Model: "model_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         [(None, 200)]             0         
_________________________________________________________________
embedding_2 (Embedding)      (None, 200, 100)          2000200   
_________________________________________________________________
flatten_4 (Flatten)          (None, 20000)             0         
_________________________________________________________________
OutputLayer (Dense)          (None, 20)                400020    
Total params: 2,400,220
Trainable params: 400,020
Non-trainable params: 2,000,200
_________________________________________________________________




MLP with the hidden layer size of 128

In [ ]:
int_sequences_input = keras.Input(shape=(200,), dtype="int64")
embedded_sequences = embedding_layer(int_sequences_input)
x = layers.Flatten()(embedded_sequences)
x = layers.Dense(128)(x)
preds = layers.Dense(len(class_names), activation="softmax", name='OutputLayer')(x)
mlp128 = keras.Model(int_sequences_input, preds)
mlp128.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 200)]             0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 200, 100)          2000200   
_________________________________________________________________
flatten_1 (Flatten)          (None, 20000)             0         
_________________________________________________________________
dense (Dense)                (None, 128)               2560128   
_________________________________________________________________
OutputLayer (Dense)          (None, 20)                2580      
Total params: 4,562,908
Trainable params: 2,562,708
Non-trainable params: 2,000,200
_________________________________________________________________


MLP with the hidden layer size of 30

In [ ]:
int_sequences_input = keras.Input(shape=(200,), dtype="int64")
embedded_sequences = embedding_layer(int_sequences_input)
x = layers.Flatten()(embedded_sequences)
x = layers.Dense(30)(x)
preds = layers.Dense(len(class_names), activation="softmax", name='OutputLayer')(x)
mlp30 = keras.Model(int_sequences_input, preds)
mlp30.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 200)]             0         
_________________________________________________________________
embedding_2 (Embedding)      (None, 200, 100)          2000200   
_________________________________________________________________
flatten_2 (Flatten)          (None, 20000)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 30)                600030    
_________________________________________________________________
OutputLayer (Dense)          (None, 20)                620       
Total params: 2,600,850
Trainable params: 600,650
Non-trainable params: 2,000,200
_________________________________________________________________


## Train the model

First, convert our list-of-strings data to NumPy arrays of integer indices. The arrays
are right-padded.

In [ ]:
x_train = vectorizer(np.array([[s] for s in train_samples])).numpy()
x_val = vectorizer(np.array([[s] for s in val_samples])).numpy()

y_train = np.array(train_labels)
y_val = np.array(val_labels)

We use categorical crossentropy as our loss since we're doing softmax classification.
Moreover, we use `sparse_categorical_crossentropy` since our labels are integers.

In [ ]:
logReg.compile(
    loss="sparse_categorical_crossentropy", optimizer="rmsprop", metrics=["acc"]
)
logReg.fit(x_train, y_train, batch_size=128, epochs=20, validation_data=(x_val, y_val))

Epoch 1/20
125/125 [==============================] - 3s 7ms/step - loss: 3.9467 - acc: 0.1738 - val_loss: 2.3518 - val_acc: 0.3513
Epoch 2/20
125/125 [==============================] - 1s 6ms/step - loss: 1.4024 - acc: 0.6027 - val_loss: 2.6496 - val_acc: 0.3423
Epoch 3/20
125/125 [==============================] - 1s 6ms/step - loss: 0.8352 - acc: 0.7695 - val_loss: 2.3790 - val_acc: 0.4134
Epoch 4/20
125/125 [==============================] - 1s 6ms/step - loss: 0.5569 - acc: 0.8541 - val_loss: 2.1932 - val_acc: 0.4219
Epoch 5/20
125/125 [==============================] - 1s 6ms/step - loss: 0.4341 - acc: 0.8969 - val_loss: 2.7365 - val_acc: 0.3878
Epoch 6/20
125/125 [==============================] - 1s 5ms/step - loss: 0.3565 - acc: 0.9259 - val_loss: 2.1982 - val_acc: 0.4546
Epoch 7/20
125/125 [==============================] - 1s 5ms/step - loss: 0.2700 - acc: 0.9469 - val_loss: 2.2547 - val_acc: 0.4376
Epoch 8/20
125/125 [==============================] - 1s 5ms/step - loss: 0.

In [ ]:
mlp128.compile(
    loss="sparse_categorical_crossentropy", optimizer="rmsprop", metrics=["acc"]
)
mlp128.fit(x_train, y_train, batch_size=128, epochs=20, validation_data=(x_val, y_val))

Epoch 1/20
125/125 [==============================] - 1s 7ms/step - loss: 16.3270 - acc: 0.1392 - val_loss: 7.1060 - val_acc: 0.3008
Epoch 2/20
125/125 [==============================] - 1s 7ms/step - loss: 3.7981 - acc: 0.4764 - val_loss: 4.5244 - val_acc: 0.3758
Epoch 3/20
125/125 [==============================] - 1s 6ms/step - loss: 1.7738 - acc: 0.6801 - val_loss: 4.2401 - val_acc: 0.3908
Epoch 4/20
125/125 [==============================] - 1s 6ms/step - loss: 1.0976 - acc: 0.7829 - val_loss: 2.9248 - val_acc: 0.4361
Epoch 5/20
125/125 [==============================] - 1s 6ms/step - loss: 0.8176 - acc: 0.8479 - val_loss: 2.9287 - val_acc: 0.4556
Epoch 6/20
125/125 [==============================] - 1s 6ms/step - loss: 0.6124 - acc: 0.8803 - val_loss: 3.5255 - val_acc: 0.4226
Epoch 7/20
125/125 [==============================] - 1s 6ms/step - loss: 0.5928 - acc: 0.8976 - val_loss: 3.0898 - val_acc: 0.4474
Epoch 8/20
125/125 [==============================] - 1s 6ms/step - loss: 0

In [ ]:
mlp30.compile(
    loss="sparse_categorical_crossentropy", optimizer="rmsprop", metrics=["acc"]
)
mlp30.fit(x_train, y_train, batch_size=128, epochs=20, validation_data=(x_val, y_val))

Epoch 1/20
125/125 [==============================] - 1s 7ms/step - loss: 7.1167 - acc: 0.1146 - val_loss: 2.8901 - val_acc: 0.3061
Epoch 2/20
125/125 [==============================] - 1s 6ms/step - loss: 2.4744 - acc: 0.4435 - val_loss: 2.5925 - val_acc: 0.3541
Epoch 3/20
125/125 [==============================] - 1s 6ms/step - loss: 1.3950 - acc: 0.6471 - val_loss: 2.1165 - val_acc: 0.4386
Epoch 4/20
125/125 [==============================] - 1s 6ms/step - loss: 0.8520 - acc: 0.7834 - val_loss: 2.3464 - val_acc: 0.4361
Epoch 5/20
125/125 [==============================] - 1s 6ms/step - loss: 0.6228 - acc: 0.8479 - val_loss: 2.3248 - val_acc: 0.4299
Epoch 6/20
125/125 [==============================] - 1s 5ms/step - loss: 0.4563 - acc: 0.8932 - val_loss: 2.2658 - val_acc: 0.4526
Epoch 7/20
125/125 [==============================] - 1s 5ms/step - loss: 0.3156 - acc: 0.9337 - val_loss: 2.3255 - val_acc: 0.4481
Epoch 8/20
125/125 [==============================] - 1s 6ms/step - loss: 0.

## Export an end-to-end model

Now, we may want to export a `Model` object that takes as input a string of arbitrary
length, rather than a sequence of indices. It would make the model much more portable,
since you wouldn't have to worry about the input preprocessing pipeline.

Our `vectorizer` is actually a Keras layer, so it's simple:

In [ ]:
string_input = keras.Input(shape=(1,), dtype="string")
x = vectorizer(string_input)
preds = mlp128(x)
end_to_end_model = keras.Model(string_input, preds)

probabilities = end_to_end_model.predict(
    [["this message is about computer graphics and 3D modeling"]]
)

In [ ]:
probabilities

array([[5.6489705e-07, 8.9249521e-01, 8.4379360e-02, 1.4128140e-02,
        1.0700257e-03, 2.5761479e-03, 4.9589849e-03, 6.4617607e-06,
        2.8695791e-05, 8.0796934e-07, 7.4193126e-06, 8.6464410e-05,
        5.2538831e-05, 6.4929409e-05, 1.3650846e-04, 2.1267282e-07,
        6.4941787e-07, 6.0097464e-06, 4.4903621e-07, 4.3670352e-07]],
      dtype=float32)

In [ ]:
class_names[np.argmax(probabilities[0])]

'comp.graphics'